In [3]:
# import
import numpy as np
import pandas as pd

import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN

# Data Preprocessing

In [4]:
# text data
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

# tokenizing
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1 # index 0 is for padding
print(tokenizer.word_index)
print(vocab_size)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}
12


In [5]:
# making data out of tokenized text
sequences = list()
for line in text.split('\n'):
    encoded = tokenizer.texts_to_sequences([line])[0] # flattening

    for i in range(1, len(encoded)): # sequence is made by at least 2 words to predict the next word
        sequence = encoded[:i+1]
        sequences.append(sequence)
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [6]:
# padding
max_len = max(len(s) for s in sequences) # max length of sequence which is 6
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

# making x and y
sequences = np.array(sequences)
X = sequences[:, :-1] # except for the last word in sequence
y = sequences[:, -1] # last word in sequence
y= to_categorical(y, num_classes=vocab_size) # one-hot encoding

print(X)
print(y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


# Modeling

In [8]:
imbedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=imbedding_dim))
model.add(SimpleRNN(units=hidden_units))
model.add(Dense(activation='softmax', units=vocab_size))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 10)          120       
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 32)                1376      
                                                                 
 dense_1 (Dense)             (None, 12)                396       
                                                                 
Total params: 1892 (7.39 KB)
Trainable params: 1892 (7.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 0s - loss: 2.5165 - accuracy: 0.0000e+00 - 300ms/epoch - 300ms/step
Epoch 2/200
1/1 - 0s - loss: 2.5004 - accuracy: 0.0000e+00 - 2ms/epoch - 2ms/step
Epoch 3/200
1/1 - 0s - loss: 2.4848 - accuracy: 0.0000e+00 - 2ms/epoch - 2ms/step
Epoch 4/200
1/1 - 0s - loss: 2.4697 - accuracy: 0.1818 - 2ms/epoch - 2ms/step
Epoch 5/200
1/1 - 0s - loss: 2.4550 - accuracy: 0.1818 - 2ms/epoch - 2ms/step
Epoch 6/200
1/1 - 0s - loss: 2.4404 - accuracy: 0.1818 - 2ms/epoch - 2ms/step
Epoch 7/200
1/1 - 0s - loss: 2.4260 - accuracy: 0.1818 - 3ms/epoch - 3ms/step
Epoch 8/200
1/1 - 0s - loss: 2.4115 - accuracy: 0.1818 - 2ms/epoch - 2ms/step
Epoch 9/200
1/1 - 0s - loss: 2.3968 - accuracy: 0.1818 - 2ms/epoch - 2ms/step
Epoch 10/200
1/1 - 0s - loss: 2.3818 - accuracy: 0.2727 - 3ms/epoch - 3ms/step
Epoch 11/200
1/1 - 0s - loss: 2.3664 - accuracy: 0.2727 - 2ms/epoch - 2ms/step
Epoch 12/200
1/1 - 0s - loss: 2.3505 - accuracy: 0.2727 - 2ms/epoch - 2ms/step
Epoch 13/200
1/1 - 0s - loss: 2.3339 - accura

In [32]:
def predict_next_word(model, tokenizer, word, n): # n is the number of words to predict
    sentence = word
    
    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([sentence])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre') # except for the last word in sequence (y)
        result = model.predict(encoded, verbose=0) # vector with the probability of each word
        result = np.argmax(result, axis=1) # index of the word with the highest probability
        
        for w, i in tokenizer.word_index.items(): # return the word with the index
            if i == result:
                break
        sentence += ' ' + w    
        
    print(sentence)

In [34]:
predict_next_word(model, tokenizer, '가는', 5)
predict_next_word(model, tokenizer, '그의', 2)
predict_next_word(model, tokenizer, '경마장에', 4)

가는 말이 고와야 오는 말이 곱다
그의 말이 법이다
경마장에 있는 말이 뛰고 있다
